# Creating & Uploading python distribution
https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container

In [128]:
!python setup.py sdist

running sdist
running egg_info
writing trainer_cpu.egg-info\PKG-INFO
writing dependency_links to trainer_cpu.egg-info\dependency_links.txt
writing requirements to trainer_cpu.egg-info\requires.txt
writing top-level names to trainer_cpu.egg-info\top_level.txt
reading manifest file 'trainer_cpu.egg-info\SOURCES.txt'
writing manifest file 'trainer_cpu.egg-info\SOURCES.txt'
running check
creating trainer-cpu-0.1
creating trainer-cpu-0.1\trainer
creating trainer-cpu-0.1\trainer\dat
creating trainer-cpu-0.1\trainer_cpu.egg-info
copying files to trainer-cpu-0.1...
copying README.md -> trainer-cpu-0.1
copying setup.py -> trainer-cpu-0.1
copying trainer\__init__.py -> trainer-cpu-0.1\trainer
copying trainer\task.py -> trainer-cpu-0.1\trainer
copying trainer\dat\__init__.py -> trainer-cpu-0.1\trainer\dat
copying trainer\dat\attack.py -> trainer-cpu-0.1\trainer\dat
copying trainer\dat\cifar_resnet18.py -> trainer-cpu-0.1\trainer\dat
copying trainer\dat\dataset.py -> trainer-cpu-0.1\trainer\dat
co

In [129]:
#!gcloud storage cp dist/dat_package-0.1.tar.gz gs://dat-package-bucket
!gsutil cp dist/trainer-cpu-0.1.tar.gz gs://dat-project-bucket/dat-package/

Copying file://dist\trainer-cpu-0.1.tar.gz [Content-Type=application/x-tar]...
/ [0 files][    0.0 B/ 15.1 KiB]                                                
/ [1 files][ 15.1 KiB/ 15.1 KiB]                                                

Operation completed over 1 objects/15.1 KiB.                                     


# Creating CustomJob
https://cloud.google.com/vertex-ai/docs/training/create-custom-job#create_custom_job-gcloud

# Operationalize Distributed Training with PyTorch on Google Cloud
https://www.youtube.com/watch?v=kXbDEmn2XOw

Pre-built-containers:
https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch

GPUs:
https://cloud.google.com/compute/docs/gpus
https://cloud.google.com/compute/gpus-pricing
https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table

In [ ]:
#!pip install google-cloud-aiplatform

In [130]:
from google.cloud import aiplatform

project_id = 'sdml-dat'
region = 'us-central1' #TODO: change to better region
bucket_name = 'dat-project-bucket'
package_uri = f'gs://{bucket_name}/dat-package/trainer-cpu-0.1.tar.gz'

# Initialize Vertex AI SDK
aiplatform.init(project=project_id, location=region, staging_bucket=bucket_name)

# Define worker pool spec
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
        },
        "replica_count": 1,  # Number of masters
        "python_package_spec": {
            "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
            "package_uris": [package_uri],
            "python_module": "trainer.task",
            "args": [
                "--gcloud=True",
                "--dataset-path=/gcs/dat-project-bucket/datasets/",
                "--output-dir=/gcs/dat-project-bucket/dat-outputs/"
            ],
        },
    }
    ,
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            # "accelerator_type": "NVIDIA_TESLA_K80",
            # "accelerator_count": 2,
        },
        "replica_count": 2,  # Number of workers
        "python_package_spec": {
            "executor_image_uri": "europe-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest",
            "package_uris": [package_uri],
            "python_module": "trainer.task",
            "args": [
                "--gcloud=True",
                "--dataset-path=/gcs/dat-project-bucket/datasets/",
                "--output-dir=/gcs/dat-project-bucket/dat-outputs/"
            ],
        },
    },
]

# Define job name
job_name = 'dat-custom-cpu-pytorch-job'

# Create a custom job
job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
)

# Launch the job
job.run(sync=True)

print(f"Job {job_name} has been launched.")

Creating CustomJob
CustomJob created. Resource name: projects/65830871515/locations/us-central1/customJobs/3523377177187844096
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/65830871515/locations/us-central1/customJobs/3523377177187844096')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3523377177187844096?project=65830871515
CustomJob projects/65830871515/locations/us-central1/customJobs/3523377177187844096 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/3523377177187844096 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/3523377177187844096 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/65830871515/locations/us-central1/customJobs/3523377177187844096 current state:
JobState.JOB_STATE_RUNNING
CustomJob projects/65830871515/locations/us-central1/customJobs/352337717718784

RuntimeError: Job failed with:
code: 8
message: "The replica workerpool0-0 ran out-of-memory and exited with a non-zero status of 9(SIGKILL). The replica workerpool1-0 ran out-of-memory and exited with a non-zero status of 9(SIGKILL). The replica workerpool1-1 ran out-of-memory and exited with a non-zero status of 9(SIGKILL). To find out more about why your job exited please check the logs: https://console.cloud.google.com/logs/viewer?project=65830871515&resource=ml_job%2Fjob_id%2F3523377177187844096&advancedFilter=resource.type%3D%22ml_job%22%0Aresource.labels.job_id%3D%223523377177187844096%22"
